# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [8]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv('../.env'); 
print('ALPHA_VANTAGE_API_KEY loaded?', bool(os.getenv('ALPHA_VANTAGE_API_KEY')))

ALPHA_VANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [9]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [ ]:
SYMBOL = 'IBM'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

In [ ]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [ ]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/Nasdaq-100'
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
    df_scrape
except Exception as e:
    print('Hi')
#     print('Scrape failed, using inline demo table:', e)
#     html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
#     soup = BeautifulSoup(html, 'html.parser')
#     rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
#     header, *data = [r for r in rows if r]
#     df_scrape = pd.DataFrame(data, columns=header)

# if 'Price' in df_scrape.columns:
#     df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
# v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Nasdaq-100 - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-featur

In [ ]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'
html = requests.get(url, timeout=30).text
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table', {'id': 'constituents'})
rows = []
for tr in table.tbody.find_all('tr')[1:]:
    tds = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
    rows.append(tds)
cols = ["Symbol", "Security", "Sector", "Product"]
df_spx = pd.DataFrame(rows, columns=cols)
df_spx

[['ADBE', 'Adobe Inc.', 'Information Technology', 'Application Software'], ['AMD', 'Advanced Micro Devices', 'Information Technology', 'Semiconductors'], ['ABNB', 'Airbnb', 'Consumer Discretionary', 'Hotels, Resorts & Cruise Lines'], ['GOOGL', 'Alphabet Inc.(Class A)', 'Communication Services', 'Interactive Media & Services'], ['GOOG', 'Alphabet Inc.(Class C)', 'Communication Services', 'Interactive Media & Services'], ['AMZN', 'Amazon', 'Consumer Discretionary', 'Broadline Retail'], ['AEP', 'American Electric Power', 'Utilities', 'Electric Utilities'], ['AMGN', 'Amgen', 'Health Care', 'Biotechnology'], ['ADI', 'Analog Devices', 'Information Technology', 'Semiconductors'], ['AAPL', 'Apple Inc.', 'Information Technology', 'Technology Hardware, Storage & Peripherals'], ['AMAT', 'Applied Materials', 'Information Technology', 'Semiconductor Equipment'], ['APP', 'AppLovin', 'Information Technology', 'Application Software'], ['ARM', 'Arm Holdings', 'Information Technology', 'Semiconductors']

,Symbol,Security,Sector,Product
0,ADBE,Adobe Inc.,Information Technology,Application Software
1,AMD,Advanced Micro Devices,Information Technology,Semiconductors
2,ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
3,GOOGL,Alphabet Inc.(Class A),Communication Services,Interactive Media & Services
4,GOOG,Alphabet Inc.(Class C),Communication Services,Interactive Media & Services
...,...,...,...,...
96,VRTX,Vertex Pharmaceuticals,Health Care,Biotechnology
97,WBD,Warner Bros. Discovery,Communication Services,Broadcasting
98,WDAY,"Workday, Inc.",Information Technology,Application Software
99,XEL,Xcel Energy,Utilities,Multi-Utilities


In [ ]:
# Save the successfully scraped NASDAQ-100 data
df_scrape = df_spx  # Use the working dataframe
_ = save_csv(df_scrape, prefix='scrape', source='wikipedia', table='nasdaq100')

OSError: Cannot save file into a non-existent directory: 'data/raw'

In [ ]:
# Validate the scraped data
v_scrape = validate(df_scrape, ['Symbol', 'Security', 'Sector', 'Product'])
print("Scrape validation:", v_scrape)
print(f"Scraped {len(df_scrape)} companies from NASDAQ-100")


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.